# Data Processing
This notebook contains the code for processing the wildfire and AQI data, namely loading the wildfire data, filtering down based on our three part criteria (defined below),generating smoke estimates (metric also defined below) and comparing our smoke estimates to that of the AQI historical data.

## Importing Libraries

In [11]:
import json

from pyproj import Transformer, Geod

#    The 'wildfire' module is a user module. 
#from wildfire.Reader import Reader as WFReader

## Loading the Data

In [13]:
with open("../data_intermediate/full_wildfires.json", "r") as file:
    full_wildfires_raw = json.load(file)

## Filter the wildfire data to include only relevant fires
Relevant fires are fires:

1.) That occured in the last 60 years of wildland fire data (1961-2021)

2.) That occured within the fire season of May 1st through October 31st

3.) Whose centroid is within 650 Miles of Norman, OK

In [14]:
# Filter to fires after 1961 
wildfires_post_1961 = [fire for fire in full_wildfires_raw if fire["attributes"]["Fire_Year"] >= 1961] #TODO update year

# Filter to something to do with fires in the fire season?
# TODO
wildfires_filtered = wildfires_post_1961



### Computing the average distance of the fire to Norman

In [15]:
### Helper Functions ###

#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average

Compute the distances, and filter to only include fires within 650 miles of Norman

In [16]:
#Define Norman Location
norman = {'city'   : 'Norman',
        'latlon' : [35.24, -97.35] }

# Filter to only include fires where the average distance is within 650 miles of Norman, OK
wildfires_near_norman = []
for index, fire in enumerate(wildfires_filtered):
    # Compute the Distance
    distance = average_distance_from_place_to_fire_perimeter(place=norman['latlon'],ring_data=fire['geometry']['rings'][0])
    #print(f"Fire '{fire['attributes']['Listed_Fire_Names'].split(',')[0]}' ({fire['attributes']['GIS_Acres']:1.2f} acres) from {fire['attributes']['Fire_Year']} was an average {distance:1.2f} miles to {norman['city']}")
    
    #Store the fire if it is within 650 miles
    if distance <= 650: 
        fire['attributes']['distance_from_norman'] = distance
        wildfires_near_norman.append(fire['attributes'])
    

Fire 'No Fire Name Provided (4)' (17062.04 acres) from 1911 was an average 1132.65 miles to Norman
Fire 'No Fire Name Provided (5)' (15096.01 acres) from 1911 was an average 1173.87 miles to Norman
Fire 'No Fire Name Provided (8)' (14533.49 acres) from 1911 was an average 1242.48 miles to Norman
Fire 'No Fire Name Provided (4)' (10394.01 acres) from 1911 was an average 1138.11 miles to Norman
Fire 'No Fire Name Provided (8)' (10307.86 acres) from 1911 was an average 1119.04 miles to Norman
Fire 'No Fire Name Provided (4)' (7249.51 acres) from 1911 was an average 1107.41 miles to Norman
Fire 'No Fire Name Provided (4)' (4819.90 acres) from 1911 was an average 1144.18 miles to Norman
Fire 'No Fire Name Provided (8)' (3788.83 acres) from 1911 was an average 1366.08 miles to Norman
Fire 'No Fire Name Provided (4)' (3374.43 acres) from 1911 was an average 1357.85 miles to Norman
Fire 'No Fire Name Provided (4)' (3215.15 acres) from 1911 was an average 1369.31 miles to Norman
Fire 'No Fire N

## Wildfire Smoke Estimates
Generate smoke estimates for Norman from all of the wildfires in our filtered dataset.

First, we assign a Smoke Impact, "SI", score that is defined as the product of two normalized features, Distance Impact ("D") * Acreage Impact ("A")
#### SI = D * A

#### 1.) Distance Impact, "D"
The Distance Impact is defined as (650 - Miles from Norman)/650. All values in our dataset take a value between 0 and 1.

Here, we assume that a fire with a distance of 0 miles from Norman has the maximum impact, and therefore gets a "Distance Impact" of 1, while a fire on the outer boundary of our cutoff (at 650 miles) has a "Distance Impact" of 0. This formula assumes that there is a linear relationship between smoke impact and distance from Norman, and that a fire past 650 miles has zero impact, both of which we know are likely not completely true, but we belive can make for useful simplifications.

#### 2.) Acreage Impact, "A"
The Acreage Impact is defined as (this_fire_acreage)/(max_fire_acreage). All values in our dataset take a value between 0 and 1.

Here, we assume that a fire with a the largest acreage has the maximum impact, and therefore gets a "Acreage Impact" of 1, while a fire with 0 acreage (i.e. not a fire) has a "Acreage Impact" of 0. Similar to above, this formula assumes that there is a linear relationship between smoke impact and fire acreage.


Overall, this formula for smoke impact makes a variety of simplifications and assumptions beyond just those listed above, including the assumption that fire duration has no impact on smoke, and additionally that fires from all directions have the same impact (but we don't have wind information, so not much we can do here).


In [17]:
#Define the smoke impace metric
def get_smoke_impact(fire, max_acres):

    # Extract distance and compute Distance Impact
    distance = fire['distance_from_norman']
    d = (650 - distance)/650

    # Extract acreage and compute Acreage Impact
    acres = fire['GIS_Acres']
    a = acres/max_acres

    # Compute combined smoke impact
    si = d*a
    
    return si

In [18]:
# Compute Smoke Impact

# Find max acres
max_acres = max(wildfires_near_norman, key=lambda x: x["GIS_Acres"])["GIS_Acres"]

# Compute Smoke Impact for all of the fires, 
for index, fire in enumerate(wildfires_near_norman):
    wildfires_near_norman[index]['Smoke_Impact'] = get_smoke_impact(fire, max_acres)

print(json.dumps(wildfires_near_norman[8], indent=4))

{
    "OBJECTID": 653,
    "USGS_Assigned_ID": 653,
    "Assigned_Fire_Type": "Wildfire",
    "Fire_Year": 1912,
    "Fire_Polygon_Tier": 1,
    "Fire_Attribute_Tiers": "1 (1), 3 (1)",
    "GIS_Acres": 4.842463010012105,
    "GIS_Hectares": 1.9596752532301926,
    "Source_Datasets": "Comb_National_NIFC_Interagency_Fire_Perimeter_History (1), Comb_National_USFS_Final_Fire_Perimeter (1)",
    "Listed_Fire_Types": "Wildfire (2)",
    "Listed_Fire_Names": "No Fire Name Provided (2)",
    "Listed_Fire_Codes": "No code provided (2)",
    "Listed_Fire_IDs": "",
    "Listed_Fire_IRWIN_IDs": "",
    "Listed_Fire_Dates": "Listed Other Fire Date(s): 2018-05-02 - NIFC DATE_CUR field (1), 2018-05-02 - REVDATE field (1)",
    "Listed_Fire_Causes": "",
    "Listed_Fire_Cause_Class": "Undetermined (2)",
    "Listed_Rx_Reported_Acres": null,
    "Listed_Map_Digitize_Methods": "Digitized Other (1)",
    "Listed_Notes": "Digitized from historic fire atlases (2)",
    "Processing_Notes": "",
    "Wildfire

In [19]:
with open("../data_intermediate/norman_wildfires_SI.json", "w") as file: #TODO Change this when running the full dataset
    json.dump(wildfires_near_norman, file, indent=4)

## Yearly Smoke Estimate

In here I want to create a new data type that has the total smoke impact for every year, where we sum the smoke impact for every fire that occured during that year